## Solución Practica

#### Objetivo: Predecir el precio del airbnb a partir del dataset proporcionado.

In [ ]:
import sys  
sys.path.insert(1, "C:/Users/ASUS/Documents/GitHub/Machine-Learning-101")

In [ ]:
from utils import plot_decision_boundary, poly_linear_regression, CM_BRIGHT

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

## 1. Carga de datos y división train/test

In [ ]:
from sklearn.model_selection import train_test_split

airbnb_data = pd.read_csv("./airbnb-listings-extract.csv", delimiter=";")
train, test = train_test_split(airbnb_data, test_size=0.3, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardar datos por separado
train.to_csv('./airbnb-listings_train.csv', sep=';', decimal='.', index=False)
test.to_csv('./airbnb-listings_test.csv', sep=';', decimal='.', index=False)

# Cargar el dataset de train
airbnb_train_data = pd.read_csv('./airbnb-listings_train.csv', sep=';', decimal='.')

In [ ]:
# Cargar el dataset de train
airbnb_train_data = pd.read_csv('./airbnb-listings_train.csv', sep=';', decimal='.')

## 2. Análisis exploratorio

### 2.1 Revisar la estructura de los datos:

In [ ]:
# Revisar las primeras filas y su contenido
train_head = airbnb_train_data.head()
train_info = airbnb_train_data.info()
train_describe = airbnb_train_data.describe(include='all')

train_head
# , train_info, train_describe

### [RUN] Eliminar columnas que considero que no aportan información relevante para predecir el precio.

In [ ]:
def eliminar_columnas (dataset): 
    return dataset.drop(['Access','Calendar last Scraped','Calendar Updated','Cancellation Policy','Description','Experiences Offered','Features', 'Host Since',
                         'First Review','Latitude','Longitude','Has Availability','Host About','Host ID','Host Location','Host Name','Host Neighbourhood','Host Picture Url','Host Response Rate',
                         'Host Response Time','Host Thumbnail Url','Host URL','Host Verifications','House Rules','ID','Interaction','Jurisdiction Names','Last Review',
                         'Last Scraped','License','Listing Url','Medium Url','Name','Neighborhood Overview','Neighbourhood Group Cleansed','Neighbourhood',
                         'Notes','Picture Url', 'Scrape ID','Space','Summary','Thumbnail Url','Transit','XL Picture Url'], axis=1)

print(f'Cantidad de columnas antes de borrar: {airbnb_train_data.shape[1]}') 
airbnb_train_data = eliminar_columnas(airbnb_train_data)
print(f'Cantidad de columnas después de borrar: {airbnb_train_data.shape[1]}')

In [ ]:
# Revisar las primeras filas y su contenido
train_head = airbnb_train_data.head()
train_info = airbnb_train_data.info()
train_describe = airbnb_train_data.describe(include='all')

train_head
# , train_info, train_describe

In [ ]:
airbnb_train_data[['Host Listings Count', 'Host Total Listings Count', 'Calculated host listings count']]

In [ ]:
print(f"Valores faltantes por columna: \n {airbnb_train_data[['Host Listings Count', 'Host Total Listings Count', 'Calculated host listings count']].isnull().sum()}")

print("\n")

correlations = airbnb_train_data[['Host Listings Count', 'Host Total Listings Count', 'Calculated host listings count', 'Price']].corr()
print("Correlaciones con el precio:")
print(correlations['Price'])

# Visualizar relación con el precio
plt.figure(figsize=(15, 5))
for i, columna in enumerate(['Host Listings Count', 'Host Total Listings Count', 'Calculated host listings count'], 1):
    plt.subplot(1, 3, i)
    plt.scatter(airbnb_train_data[columna], airbnb_train_data['Price'], alpha=0.5)
    plt.title(f'Relación: {columna} vs Price')
    plt.xlabel(columna)
    plt.ylabel('Price')
plt.tight_layout()
plt.show()

### Analizar Country y Country Code

In [ ]:
airbnb_train_data["Country Code"].unique(), airbnb_train_data["Country"].unique()

### [RUN] Eliminar columnas ronda 2

In [ ]:
def eliminar_columnas_ronda_dos (dataset): 
    return dataset.drop(['Host Total Listings Count', 'Host Listings Count', 'Country Code'], axis=1)

print(f'Cantidad de columnas antes de borrar: {airbnb_train_data.shape[1]}') 
airbnb_train_data = eliminar_columnas_ronda_dos(airbnb_train_data)
print(f'Cantidad de columnas después de borrar: {airbnb_train_data.shape[1]}')

### Conclusiones

* **ID, Listing Url, Scrape ID, Host ID, Host URL, Picture Url, Thumbnail Url, etc.**, son identificadores únicos o enlaces que no aportan información valiosa.
* **Last Scraped, Name, Summary, Description, Experiences Offered, Neighborhood Overview, Notes, Street, etc.**, contienen texto descriptivo que probablemente no tenga un impacto significativo en este caso.
* **First Review y Last Review**: contiene fechas de la primera y última reseña que no aportan valor significativo.
* **Calendar Updated y Calendar last Scraped**: proveen información de cuándo se actualizó o descargó el calendario de disponibilidad y esto no es relevante para el calculo del precio.
* **Latitud y Longitud:** Es redudante porque ya tenemos Geolocation.
* **Availability 30, Availability 60, Availability 90, Availability 365**: considero que no son relevantes para calcular el precio. Estan mas enfocadas a proveer informacion sobre disponibilidad en diferentes intervalos de tiempo.
* **Host Total Listings Count y Listings Count**: Tienen correlaciones similares con el precio y probablemente son redundantes comparada con Calculated Host Listings Count, pero está podria representar una versión derivada más precisa.
* **Country Code:** Procedemos a eliminar esta columa porque es redundante con Country y además contiene registros con codigo "It" y "IT" así que nos ahorramos ese procesamiento a mayusculas. 

### 2.2 Detectar valores faltantes y outliers

### Valores faltantes

In [ ]:
airbnb_train_data.isnull().sum()

In [ ]:
def mostrar_porcentaje_valores_faltantes(dataset):
    porcentaje_valores_faltantes = dataset.isnull().mean() * 100
    print("Porcentaje de valores faltantes por columna:")
    print(porcentaje_valores_faltantes[porcentaje_valores_faltantes > 0].sort_values(ascending=False))

mostrar_porcentaje_valores_faltantes(airbnb_train_data)

### [RUN] Se descartaran las columnas que tienen mas del 70% de valores faltantes:
- Host Acceptance Rate (99.76%)
- Square Feet (96.04%)
- Monthly Price (75.82%)
- Weekly Price (75.71%)

In [ ]:
def eliminar_columnas_na (dataset): 
    return dataset.drop(['Host Acceptance Rate', 'Square Feet', 'Monthly Price', 'Weekly Price'], axis=1)

print(f'Cantidad de columnas antes de borrar: {airbnb_train_data.shape[1]}')
airbnb_train_data = eliminar_columnas_na(airbnb_train_data)
print(f'Cantidad de columnas después de borrar: {airbnb_train_data.shape[1]}')

mostrar_porcentaje_valores_faltantes(airbnb_train_data)

## Imputación de valores

### [RUN] Imputare los valores nulos de tipo numerico con la mediana ya que no quiero que se vean afectado por los outliers.

In [ ]:
airbnb_train_data[['Bathrooms', 'Beds', 'Bedrooms']].describe()

In [ ]:
def obtener_columnas_numericas():
    return airbnb_train_data.select_dtypes(include=['float64', 'int64']).columns

for columna in obtener_columnas_numericas():
    airbnb_train_data[columna] = airbnb_train_data[columna].fillna(airbnb_train_data[columna].median())

mostrar_porcentaje_valores_faltantes(airbnb_train_data)

In [ ]:
airbnb_train_data[['Security Deposit']].describe()

### [RUN] Imputare los valores nulos de Security Deposit con 0

In [ ]:
airbnb_train_data['Security Deposit'] = airbnb_train_data['Security Deposit'].fillna(0)
mostrar_porcentaje_valores_faltantes(airbnb_train_data)

#### Validar porcentaje de filas restantes con valores nulos

In [ ]:
columnas_con_valores_nulos = [
    'State', 'Zipcode', 'Market', 
    'City', 'Amenities', 'Country'
]
datase_filas_eliminadas = airbnb_train_data.dropna(subset=columnas_con_valores_nulos)
total_filas_antes = airbnb_train_data.shape[0]
total_filas_despues = datase_filas_eliminadas.shape[0]

print(
    f'Original: {total_filas_antes} // '
    f'Modificado: {total_filas_despues}\nDiferencia: {total_filas_antes - total_filas_despues}'
)
print(f'Variación: {((total_filas_antes - total_filas_despues)/total_filas_antes)*100:2f}%')

### [RUN] Imputare los valores nulos de las filas restantes con la moda para no perder el 5% de la información.

In [ ]:
airbnb_train_data[columnas_con_valores_nulos].mode()

In [ ]:
# Rellenar valores nulos con la moda
for columna in columnas_con_valores_nulos:
    moda = airbnb_train_data[columna].mode()[0]
    airbnb_train_data[columna] = airbnb_train_data[columna].fillna(moda)

# Verificar si quedan valores nulos
print("Valores nulos después de la imputación:")
print(airbnb_train_data[columnas_con_valores_nulos].isnull().sum())

In [ ]:
airbnb_train_data.info()

In [ ]:
airbnb_train_data.isnull().sum()

### Codificación de variables categóricas

In [ ]:
columnas_tipo_objeto = airbnb_train_data.select_dtypes(include='object').columns

for columna in columnas_tipo_objeto:
    print(f"{columna}: {airbnb_train_data[columna].nunique() }")

# airbnb_train_data["Bed Type"].unique(), airbnb_train_data["Room Type"].unique(), airbnb_train_data["Property Type"].unique()
airbnb_train_data["Market"].unique()

In [ ]:
airbnb_train_data['Street'].head(5), airbnb_train_data['Market'].head(5)

### [RUN] Codificando variables

In [ ]:
from sklearn.preprocessing import LabelEncoder

def one_hot_encode(dataset, columnas):
    """Aplica One-Hot Encoding a las columnas seleccionadas."""
    return pd.get_dummies(dataset, columns=columnas, drop_first=True)

def label_encode(dataset, columnas):
    """Aplica Label Encoding a las columnas seleccionadas."""
    for columna in columnas:
        le = LabelEncoder()
        dataset[columna] = le.fit_transform(dataset[columna])
    return dataset

def procesar_amenidades(dataset, columna):
    """Crea una nueva columna con el conteo de amenidades."""
    dataset['Amenities_Count'] = dataset[columna].apply(lambda x: len(x.split(',')))
    return dataset

airbnb_encoded = airbnb_train_data.copy()

# Aplicar One-hot
airbnb_encoded = one_hot_encode(airbnb_encoded, ['Room Type', 'Property Type', 'Bed Type', 'Market'])

# Aplicar Label Encoding
airbnb_encoded = label_encode(airbnb_encoded, ['Neighbourhood Cleansed', 'Zipcode'])

# Aplicar conteo de características a Amenities
airbnb_encoded = procesar_amenidades(airbnb_encoded, 'Amenities')

In [ ]:
def eliminar_columnas_codificadas(dataset, columnas_a_eliminar):
    """
    Elimina las columnas originales que fueron codificadas o procesadas
    con el fin de evitar duplicidad en el dataset.
    """
    dataset.drop(columns=columnas_a_eliminar, inplace=False, errors='ignore')
    return dataset

print(f"antes: {airbnb_encoded.shape[1]}")

# Eliminar columnas codificadas
airbnb_encoded = eliminar_columnas_codificadas(airbnb_encoded, ['Room Type', 'Property Type', 'Bed Type', 'Market', 
                        'Neighbourhood Cleansed', 'Zipcode', 'Amenities'])

# Verificar las columnas restantes
print(f"despues: {airbnb_encoded.shape[1]}")

## Conclusiones
* Ninguna variable categórica parece tener un orden logico así que concluyo que todas son nominales.
* Street y Geolocation: procedemos a descartarlas porque tienen un numero excesivo de categorías. No son utiles para predecir el precio ya que no se observan patrones repetibles.
* Amenities tiene un número muy alto de valores unicos así que procederemos a crear una variable derivada para contar la cantidad de amenidades. Luego analizar si esto es util para predecir el precio.
* Room Type, Property Type y Bed Type pueden tener un impacto alto en el precio.

### Identificar outliers en columnas numéricas

In [ ]:
def grafica_frecuencia_por_columnas(activarBreak = True):
    for columna in obtener_columnas_numericas():
        plt.figure(figsize=(5, 3))  # Ajusta el tamaño de cada figura individual
        airbnb_train_data[columna].plot.hist(alpha=0.5, bins=25, grid=True)
        plt.xlabel(columna)
        plt.ylabel("Frecuencia")
        plt.title(f"Distribución de {columna}")
        plt.show()
        
        if activarBreak: 
            break

grafica_frecuencia_por_columnas(activarBreak = True)

In [ ]:
# Cantidad de registros con 0 baños
print(f"0 Baños: {airbnb_train_data[(airbnb_train_data['Bathrooms'] == 0)].shape[0]}")

# Cantidad de registros con 0 Dormitorios
print(f"0 Dormitorios: {airbnb_train_data[(airbnb_train_data['Bedrooms'] == 0)].shape[0]}")

# Cantidad de registros con 0 Camas
print(f"0 Camas: {airbnb_train_data[(airbnb_train_data['Beds'] == 0)].shape[0]}")

# Cantidad de registros con 0 Precio
print(f"0 Precio: {airbnb_train_data[(airbnb_train_data['Price'] == 0)].shape[0]}")

filtro_0_dormitorios = airbnb_train_data[(airbnb_train_data['Bedrooms'] == 0)][['Bathrooms', 'Beds', 'City', 'Property Type', 'Price']]
filtro_ordenado = filtro_0_dormitorios.sort_values(by='Bathrooms', ascending=True)
filtro_ordenado

In [ ]:
# print(airbnb_train_data['Minimum Nights'].describe())
# print(airbnb_train_data['Maximum Nights'].describe())

# print(airbnb_train_data[airbnb_train_data['Minimum Nights'] > 30][['Minimum Nights']].unique())
# airbnb_train_data['Minimum Nights'].unique()

# valores_frecuencia_ordenados = airbnb_train_data['Minimum Nights'].value_counts().sort_index()
# valores_frecuencia_ordenados

percentil_99_min = airbnb_train_data['Minimum Nights'].quantile(0.99)
print("Percentil 99 para Minimum Nights:", percentil_99_min)

percentil_99_max = airbnb_train_data['Maximum Nights'].quantile(0.99)
print("Percentil 99 para Maximum Nights:", percentil_99_max)

print(f"Outliers en Minimum Nights: {airbnb_train_data[airbnb_train_data['Minimum Nights'] > 30].shape[0]}")
print(f"Outliers en Maximum Nights: {airbnb_train_data[airbnb_train_data['Maximum Nights'] > 1130].shape[0]}")

In [ ]:
def grafico_minimo_maximo_noches():
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    airbnb_train_data['Minimum Nights'].plot.hist(bins=30, alpha=0.7)
    plt.title("Minimum Nights (Límite en 30)")
    
    plt.subplot(1, 2, 2)
    airbnb_train_data['Maximum Nights'].plot.hist(bins=30, alpha=0.7)
    plt.title("Maximum Nights (Límite en 1130)")
    
    plt.tight_layout()
    plt.show()

grafico_minimo_maximo_noches()

### [RUN] Ajustar minimo y maximo de noches con los maximos para evitar perder información

In [ ]:
airbnb_train_data['Minimum Nights'] = airbnb_train_data['Minimum Nights'].clip(upper=30)
airbnb_train_data['Maximum Nights'] = airbnb_train_data['Maximum Nights'].clip(upper=1130)

print(f"Minimum: {airbnb_train_data['Minimum Nights'].max()}")
print(f"Maximum: {airbnb_train_data['Maximum Nights'].max()}")

grafico_minimo_maximo_noches()

### [RUN] Eliminar registros con 0 baño

In [ ]:
airbnb_train_data = airbnb_train_data[airbnb_train_data['Bathrooms'] != 0]

# Cantidad de registros con 0 baños
print(f"0 Baños: {airbnb_train_data[(airbnb_train_data['Bathrooms'] == 0)].shape[0]}")

### Conclusiones

* Se descartaron los 44 registros que tienen 0 baño.
* Se mantienen los 707 registros que tienen 0 Dormitorio porque pueden ser apartaestudios.
* La gran mayoría de los valores de Minimum Nights están concentrados en valores pequeños con un percentil 99 de 28 noches. Así que se ajustaron los registros con más de 30 días para que tomen el valor maximo y evitar perder información. En este  caso, cualquier valor superior a 30 será reemplazado por 30.
* Se hizo algo similar con el Maximum Nights. Cualquier valor superior a 1.130 será reemplazado por 1130.

### **[RUN] Eliminar los outliers de algunas variables numéricas

In [ ]:
# def eliminar_outliers(data, columnas_numericas):
#     """
#     Elimina los outliers de las columnas numéricas en un DataFrame utilizando el rango intercuartílico (IQR).
#     """
#     coeficiente_multiplicador = 1.5
    
#     for col in columnas_numericas:
#         q1 = data[col].quantile(0.25)  # Primer cuartil (25%)
#         q3 = data[col].quantile(0.75)  # Tercer cuartil (75%)
#         iqr = q3 - q1  # Rango intercuartílico
        
#         # Limites para detectar outliers
#         limite_inferior = q1 - coeficiente_multiplicador * iqr
#         limite_superior = q3 + coeficiente_multiplicador * iqr
        
#         # Filtrar los datos dentro de los límites
#         data = data[(data[col] >= limite_inferior) & (data[col] <= limite_superior)]
#     return data

# columnas_para_remover_outliers=['Security Deposit', 'Cleaning Fee', 'Guests Included', 'Extra People',
#                                 'Minimum Nights', 'Maximum Nights', 'Number of Reviews']
# airbnb_train_data = eliminar_outliers(airbnb_train_data, columnas_para_remover_outliers)
# grafica_frecuencia_por_columnas(activarBreak = False)

In [ ]:
airbnb_train_data.shape

#### Relación entre `Price` y las columnas numéricas

In [ ]:
plt.figure(figsize=(15, 5))
for i, columna in enumerate(obtener_columnas_numericas(), 1):
    if columna != 'Price':
        airbnb_train_data.plot(kind = 'scatter',x=columna,y = 'Price')
        plt.title(f"Relación: {columna} vs Price")
        plt.xlabel(columna)
        plt.ylabel('  ($)')
        break
plt.show()

### Conclusiones
* Existe una relación positiva entre Accommodates y Price, alojamientos con mayor capacidad tienden a ser más caros.
    * En alojamientos con mayor capacidad (más de 8 personas) el precio tiende a ser consistentemente más alto.
* Tambien hay una relación positiva entre Bathrooms y Price, más baños generalmente estan asociados con alojamientos más caros.
    * El precio generalmente aumenta con el número de baños pero hay menos alojamientos con más de 3 baños.
    * Algunos alojamientos con 2-3 baños tienen precios muy altos, lo mas probablemente es que sea debido a factores externos.
* Los precios más altos están asociados con alojamientos que tienen puntuaciones de limpieza de 9 o 10.
* No veo relación fuerte de los campos Review Scores Communication y Review Scores Checkin con el precio.

### [RUN] Eliminar columnas redudantes ronda 3

In [ ]:
def eliminar_columnas_ronda_tres (dataset): 
    return dataset.drop(['Review Scores Communication', 'Review Scores Checkin', 'Availability 30', 'Availability 60', 'Availability 90'], axis=1)

print(f'Cantidad de registros: {airbnb_train_data.shape[0]}') 
print(f'Cantidad de columnas antes de borrar: {airbnb_train_data.shape[1]}') 
airbnb_train_data = eliminar_columnas_ronda_tres(airbnb_train_data)
print(f'Cantidad de columnas después de borrar: {airbnb_train_data.shape[1]}')

In [ ]:
grafica_frecuencia_por_columnas(activarBreak = True)

In [ ]:
# airbnb_train_data.corr() # matriz de correlación
numerical_data = airbnb_train_data.select_dtypes(include=['float64', 'int64'])

# Calcular la matriz de correlación
numerical_data.corr()

In [ ]:
import seaborn as sns

numerical_data = airbnb_train_data.select_dtypes(include=['float64', 'int64'])

# Compute the correlation matrix
corr = np.abs(numerical_data.drop(['Price'], axis=1).corr())

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(6, 5))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,vmin = 0.0, vmax=1.0, center=0.5,
            linewidths=.1, cmap="YlGnBu", cbar_kws={"shrink": .8})

plt.show()

# 3. Preprocesamiento:

# 4. Modelado, cross-validation y estudio de resultados en train y test